In [1]:
"""Import all the relevant libraries."""

from mkd_blockchain import MKDBlockchain
from blockchain_utils import BlockchainUtils
import block
from pubsub import pub
from node import Node
from transaction import Transaction
from transaction_pool import TransactionPool
from wallet import Wallet
from Crypto.PublicKey import RSA
import pprint
import random
import time
from mkdtree import kdtree


In [2]:
"""Initialize Nodes with Genesis Block"""

print('START')
MAX_NODES = 9999
num_nodes = 10
forging_interval = 10  # Time interval.  10 units of time
nodes = [] #list of all nodes
blockchain_dimensions = 4
genesis_node_id = int(num_nodes/2)

#itterate through and initialize each node
then = time.time()
for i in range(num_nodes):
    new_node = Node(i+1, 0)
    #new_node.blockchain = mkd_blockchain
    new_node.start_listener('c0')
    nodes.append(new_node)
now = time.time()-then
print(f'block init time @ {num_nodes} nodes: {now}')


genesis_forger = nodes[genesis_node_id-1].wallet.public_key_string()
mkd_blockchain = MKDBlockchain(blockchain_dimensions, genesis_node_id, genesis_forger)


publisher = nodes[genesis_node_id-1]

start_broadcast = time.time()
publisher.publish(mkd_blockchain)
completed_broadcast = time.time() - start_broadcast
print(f'time to complete broadcast to {num_nodes} nodes: {completed_broadcast}')


START
block init time @ 10 nodes: 24.462502002716064
c1 received payload: <mkd_blockchain.MKDBlockchain object at 0x0000021CB0436880>
c2 received payload: <mkd_blockchain.MKDBlockchain object at 0x0000021CB0436880>
c3 received payload: <mkd_blockchain.MKDBlockchain object at 0x0000021CB0436880>
c4 received payload: <mkd_blockchain.MKDBlockchain object at 0x0000021CB0436880>
c5 received payload: <mkd_blockchain.MKDBlockchain object at 0x0000021CB0436880>
c6 received payload: <mkd_blockchain.MKDBlockchain object at 0x0000021CB0436880>
c7 received payload: <mkd_blockchain.MKDBlockchain object at 0x0000021CB0436880>
c8 received payload: <mkd_blockchain.MKDBlockchain object at 0x0000021CB0436880>
c9 received payload: <mkd_blockchain.MKDBlockchain object at 0x0000021CB0436880>
c10 received payload: <mkd_blockchain.MKDBlockchain object at 0x0000021CB0436880>
time to complete broadcast to 10 nodes: 0.0020241737365722656


In [5]:
"""Attempting to add a second block to the kdtree_blockchain"""

attempt_node_id = 6
attempt_node = nodes[attempt_node_id-1]

attempt_node.test_forge()

kdtree.visualize(attempt_node.blockchain.blocks)

attempt_node_id = 3
attempt_node = nodes[attempt_node_id-1]

attempt_node.test_forge()

kdtree.visualize(attempt_node.blockchain.blocks)

attempt_node_id = 6
attempt_node = nodes[attempt_node_id-1]

attempt_node.test_forge()

kdtree.visualize(attempt_node.blockchain.blocks)

attempt_node_id = 3
attempt_node = nodes[attempt_node_id-1]

attempt_node.test_forge()

kdtree.visualize(attempt_node.blockchain.blocks)


I am the forger: 
Traversed block list: [<KDNode - Block([5, 0, 0, 1626043929.3614566], 0, [])>, <KDNode - Block([6, 927, 330, 1626043929.3764558], d895514a98b5612590804162b5f75b00bcecd501, [])>]
Traversed block list: [<KDNode - Block([5, 0, 0, 1626043929.3614566], 0, [])>, <KDNode - Block([6, 927, 330, 1626043929.3764558], d895514a98b5612590804162b5f75b00bcecd501, [])>, <KDNode - Block([6, 195, 197, 1626043929.3994558], 754b693ce24994faa9009f06fc24046c158dc440, [])>]


                        Block([5, 0, 0, 1626043929.3614566], 0, [])                    

      Block([3, 479, 617, 1626043929.3884559], d895514a98b5612590804162b5f75b00bcecd501, []) Block([6, 927, 330, 1626043929.3764558], d895514a98b5612590804162b5f75b00bcecd501, []) 

                           Block([3, 950, 803, 1626043929.4124568], be41937dba43108e8fb5c982da6304245e03be38, []) Block([6, 195, 197, 1626043929.3994558], 754b693ce24994faa9009f06fc24046c158dc440, [])                      

                              

In [4]:
#Opens the data set file and generates nodes based on the provided data.
#file is comma delimited in the following format: time-nodeID-xcoor-ycoor-miner-region-prevregion
then = time.time()
with open('node_data.txt') as f:
    lines = f.readlines() # list containing lines of file
    i = 0

    for line in lines:
        j = i%80
        print(j)
        parts = line.split(',')
        moving_node = nodes[j]

        old_topic = 'c'+str(moving_node.cluster_id).strip()
        new_topic = 'c' + str(parts[5]).strip()

        moving_node.coords = [parts[2], parts[3]]

        moving_node.cluster_id = parts[5]
        moving_node.move_listener(old_topic, new_topic)

        #merge right after movement

        #variable that sends transaction if miner==1
        if parts[4] == 1:
            # initiate transaction, save it to pool and broadcast.
            x=1

        # if time to forge then forge and broadcast, needs to scan and perform all clusters
            # if transaction pool not empty then forge and broadcast


        # move_listener(node[i % num_nodes].node_listener, "c" + str(parts[6]).strip(), "c" + str(parts[5]).strip())

        i += 1
now = time.time() - then
print(f'node movement time @ {num_nodes} nodes: {now}')

0
1
2
3
4
5
6
7
8
9
10


IndexError: list index out of range